##  Install stable baselines3 and import dependencies

https://stable-baselines3.readthedocs.io/en/master/

the algorithm is based in this well explined web page 

In [ ]:
!pip install stable-baselines3[extra]

In [4]:
import os
import gym  # is a open AI library to create enviroments
from stable_baselines3 import PPO  # PPO is the type of RL algorithm that we are gonna use
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy # to test how the model is actually performing

In [6]:
environment_name = "CartPole-v0" # this is the well know balance robot 

In [7]:
 env = gym.make(environment_name, render_mode="human") # 2 render mode "human show image" "rgb_arraw shows the values"

C:\Users\Ferna\AppData\Roaming\Python\Python310\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [8]:
episodes = 5
for episode in range(1, episodes+1):  # we're gonna check 5 times
    state = env.reset() # reset the env so we have a initial set of observations 
    done = False
    score = 0 
    
    while not done:
        env.render() # visualise the enviroment 
        action = env.action_space.sample() # take an action randomicaly
        n_state, reward, done, info,s = env.step(action) # apply an action to the enviroment 
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() # close down the render frame

Episode:1 Score:11.0
Episode:2 Score:27.0
Episode:3 Score:16.0
Episode:4 Score:17.0
Episode:5 Score:18.0


##### some explication of the comands 

In [11]:
env.reset()  #  this reset the enviroment and give a set of observations then we pass to RL and obtain a reward and max it 
env.action_space  # this say the numbers of actions 
print(env.action_space.sample()) # this choose one action among the availables 
print(env.observation_space.sample()) # this show show the values of the enviroment 
print(env.step(1)) # return a set of observations, reward, and true of false if whether or not a episode is done 

(array([ 0.01928254, -0.3557433 ,  0.03629818,  0.59017646], dtype=float32),
 1.0,
 False,
 False,
 {})

### undertanding the enviroment 

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

    | Num | Action                 |
    |-----|------------------------|
    | 0   | Push cart to the left  |
    | 1   | Push cart to the right |
    
    
    | Num | Observation           | Min                 | Max               |
    |-----|-----------------------|---------------------|-------------------|
    | 0   | Cart Position         | -4.8                | 4.8               |
    | 1   | Cart Velocity         | -Inf                | Inf               |
    | 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
    | 3   | Pole Angular Velocity | -Inf                | Inf               |
    

In [25]:
env.action_space # Notice that the env is dicrete it is importat to choose the RL algorithm 

Discrete(2)

In [26]:
env.action_space.sample() # choose one action

0

In [27]:
env.observation_space 

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [29]:
env.observation_space.sample() # to see the values ob the environment

array([ 9.8566480e-02,  1.8670842e+38, -2.0434605e-01,  3.1199025e+38],
      dtype=float32)

## Training RL model

In [30]:
log_path = os.path.join('Training','logs')  # first, you need to create the forder 

In [53]:
# instanciate an algorithm especificaly a agent 
env = gym.make(environment_name)#, render_mode="human")
env = DummyVecEnv([lambda: env]) # wrapped the env inside of the dummy vectorice enviroment
model = PPO('MlpPolicy',env, verbose=1,tensorboard_log=log_path) # MlpPolicy= multilayer perceptron model policy,
# a policy is how the agent operate in the env, tensorboard is to check the metrix

Using cpu device


In [35]:
# to check the PPO arguments
PPO??

In [54]:
model.learn(total_timesteps=20000)  # train the agent

Logging to Training\Saved Models\PPO_Cartpole_model\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1438 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 948         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008065789 |
|    clip_fraction        | 0.0922      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.027       |
|    learning_rate        | 0.0003      |
|    loss                 | 9.19        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.014      |
|    value_loss           | 57.4        |
------------------

#### Save the model 

In [55]:
PPO_Path = os.path.join('Training','Saved Models','PPO_Cartpole_model')

In [56]:
model.save(PPO_Path)

How to delete the model and reload 

In [46]:
del model

In [47]:
model = PPO.load(PPO_Path, env=env)

#### Evaluation

In [58]:
env = gym.make(environment_name, render_mode="human")
evaluate_policy(model, env, n_eval_episodes=10, render=True) # render is to visualize

(200.0, 0.0)

evaluate policy is 2 number (0 ,0 ) first tell the avr reward and second the standar deviation in that reward.
the reward dor CartPole is calculate as 1 point for every step that the pole remains upright (max=200), if the pole is more that 15 degrees from the vertical or the cart moves more thatn 2.4 units from the center the episode ends

In [59]:
env.close()

### Test the model

instead to evaluate with a encapsulate function as before let's create a evaluation where we use the trained agent 

In [67]:
env = gym.make(environment_name, render_mode="human")

In [82]:
episodes = 5
for episode in range(1, episodes+1):  
    observations,_ = env.reset() 
    done = False
    score = 0 
    
    while not done:
        env.render() 
        action,_ = model.predict(observations) # here for the action we use the prediction of our trained model
        observations, reward, done, info,_ = env.step(action) 
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() 

error: display Surface quit

### Callbacks

we're gonna specify a reward threshold so this means that our training is going to stop once it hits a certain benchmark: useful
when you have a large models and you want to stop them before start getting unstable.  good part is that automaticaly we will save our best model

In [83]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os sssasas  ss

In [85]:
save_path = os.path.join('Training', 'Saved Models')

In [94]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)  # to stop the training when reach certain reward
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)    
# the eval is triggered after each training run. env= environment, callback_on_new= everi time that there is a new best model call the other callback
# eval_freq = how frecuenly we want to run the evaluation callback, best_model= save the model every time that there are a new best model

# Now asociate this callbacks to a model

In [95]:
env = gym.make(environment_name)
model = PPO('MlpPolicy',env, verbose=1,tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [96]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Saved Models\PPO_Cartpole_model\PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.8     |
|    ep_rew_mean     | 23.8     |
| time/              |          |
|    fps             | 1350     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.8        |
|    ep_rew_mean          | 25.8        |
| time/                   |             |
|    fps                  | 931         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007717683 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained

### Now if we want to use different NN to train the model "change the policy"

In [100]:
# change the number of neuronas and layers 
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])] # for value funcion and pi we ser 4 layes of 128 neurons

In [102]:
env = gym.make(environment_name)
model = PPO('MlpPolicy',env, verbose=1,tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Ferna\anaconda3\lib\site-packages\stable_baselines3\common\policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [103]:
model.learn(total_timesteps=20000, callback=eval_callback) # this is using a new NN architecture so it is a new policy

Logging to Training\Saved Models\PPO_Cartpole_model\PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.6     |
|    ep_rew_mean     | 23.6     |
| time/              |          |
|    fps             | 918      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30          |
|    ep_rew_mean          | 30          |
| time/                   |             |
|    fps                  | 521         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014212115 |
|    clip_fraction        | 0.248       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 159      |
|    ep_rew_mean     | 159      |
| time/              |          |
|    fps             | 462      |
|    iterations      | 10       |
|    time_elapsed    | 44       |
|    total_timesteps | 20480    |
---------------------------------


### Train with other algorithm 

In [104]:
from stable_baselines3 import DQN

In [105]:
model = DQN('MlpPolicy',env, verbose=1,tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [106]:
model.learn(total_timesteps=20000) # now we are training with diferent algorithm

Logging to Training\Saved Models\PPO_Cartpole_model\DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21       |
|    ep_rew_mean      | 21       |
|    exploration_rate | 0.96     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2087     |
|    time_elapsed     | 0        |
|    total_timesteps  | 84       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23.9     |
|    ep_rew_mean      | 23.9     |
|    exploration_rate | 0.909    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3282     |
|    time_elapsed     | 0        |
|    total_timesteps  | 191      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24.2     |
|    ep_rew_mean      | 24.2     |
|    exploration_rate | 0.862   

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 92       |
|    fps              | 5104     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2062     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 96       |
|    fps              | 5081     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2153     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 180      |
|    fps              | 4990     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4029     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.1     |
|    ep_rew_mean      | 22.1     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 184      |
|    fps              | 4957     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4096     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.2     |
|    ep_rew_mean      | 22.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.9     |
|    ep_rew_mean      | 20.9     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 268      |
|    fps              | 4712     |
|    time_elapsed     | 1        |
|    total_timesteps  | 5875     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.6     |
|    ep_rew_mean      | 20.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 272      |
|    fps              | 4702     |
|    time_elapsed     | 1        |
|    total_timesteps  | 5938     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.3     |
|    ep_rew_mean      | 20.3     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.4     |
|    ep_rew_mean      | 21.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 356      |
|    fps              | 4536     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7769     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.9     |
|    ep_rew_mean      | 21.9     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 360      |
|    fps              | 4541     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7896     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22       |
|    ep_rew_mean      | 22       |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.8     |
|    ep_rew_mean      | 21.8     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 4586     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11712    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.8     |
|    ep_rew_mean      | 21.8     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 4591     |
|    time_elapsed     | 2        |
|    total_timesteps  | 11789    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.6     |
|    ep_rew_mean      | 21.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.6     |
|    ep_rew_mean      | 21.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 620      |
|    fps              | 4719     |
|    time_elapsed     | 2        |
|    total_timesteps  | 13556    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.4     |
|    ep_rew_mean      | 21.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 624      |
|    fps              | 4726     |
|    time_elapsed     | 2        |
|    total_timesteps  | 13653    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.9     |
|    ep_rew_mean      | 21.9     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.6     |
|    ep_rew_mean      | 22.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 708      |
|    fps              | 4846     |
|    time_elapsed     | 3        |
|    total_timesteps  | 15576    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.5     |
|    ep_rew_mean      | 22.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 712      |
|    fps              | 4849     |
|    time_elapsed     | 3        |
|    total_timesteps  | 15660    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.6     |
|    ep_rew_mean      | 21.6     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 796      |
|    fps              | 4876     |
|    time_elapsed     | 3        |
|    total_timesteps  | 17434    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.5     |
|    ep_rew_mean      | 21.5     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 800      |
|    fps              | 4881     |
|    time_elapsed     | 3        |
|    total_timesteps  | 17526    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.3     |
|    ep_rew_mean      | 21.3     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.1     |
|    ep_rew_mean      | 22.1     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 884      |
|    fps              | 4926     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19420    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.2     |
|    ep_rew_mean      | 22.2     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 888      |
|    fps              | 4931     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19516    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes       